In [5]:
import cvxpy as cp
import numpy as np
from itertools import chain, combinations
import scipy.io

In [2]:
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [24]:
# Constants
Ly = 2 # number of antennas at the AP
U = 2 # number of users
N = 64 # number of FFT tones
sigma_square = 1.0/9800000 # noise power
Lx = [1] * U  # Lx(u) = 2 for all u (number of antennas at each user)
bmin = [170.1193, 176.8150, 244.5390]  # bmin(u) = 10 for all u (minimum rate requirement for each user)
# decoding order
# pi_inv = [0, 1, 2]  

In [20]:
# # Example H(u, n): random complex matrices of dimension Ly * Lx(u); one for each user u and each FFT tone n
# H = [[np.random.randn(Ly, Lx[u]) + 1j*np.random.randn(Ly, Lx[u]) for n in range(N)] for u in range(U)]
# # make H(u,n) unit magnitude for each user u and each FFT tone n, and then divide by square root of noise power
# H = [[H[u][n] / np.linalg.norm(H[u][n]) / np.sqrt(sigma_square) for n in range(N)] for u in range(U)]



In [21]:
# # make channel of user 2 stronger by multiplying it by 10
# H[1] = [10 * H[1][n] for n in range(N)]
# # change the minimum rate requirement of user 1 to 20
# bmin[0] = 20

In [25]:
# load H from H.mat file
H_nd = scipy.io.loadmat('../H.mat')['H']
H_nd = np.array(H_nd, dtype=complex)
print (H_nd.shape)
H = [[H_nd[:,u,n].reshape((Ly, Lx[u])) for n in range(N)] for u in range(U)]
print (len(H), len(H[0]), H[0][0].shape)
H = [[H[u][n] / np.sqrt(sigma_square) for n in range(N)] for u in range(U)]
print (len(H), len(H[0]), H[0][0].shape)

(2, 3, 64)
3 64 (2, 1)
3 64 (2, 1)


In [26]:
# Variables
Rxx = [[cp.Variable((Lx[u], Lx[u]), PSD=True) for n in range(N)] for u in range(U)]
b = [[cp.Variable(nonneg = True) for n in range(N)] for u in range(U)]
# pi_inv = cp.Variable(U, integer=True)  # Permutation variable
# thetas = cp.Variable(U, nonneg=True)

In [27]:
# Constraints
constraints = []
b_constraints = []

# Constraint: sum of b(u, n) over n for each u is at least bmin(u)
for u in range(U):
    constraint = sum(b[u][n] for n in range(N)) >= bmin[u]
    constraints.append(constraint)
    b_constraints.append(constraint)  # Keep track of these constraints for dual variables

# Constraint: b(u, n) >= 0 for all u, n
for u in range(U):
    for n in range(N):
        constraints.append(b[u][n] >= 0)

# Constraint: sum of b(u, n) over u for each n is limited by the log-det expression
for n in range(N):
    # Compute the matrix sum inside the determinant
    matrix_sum = sum(cp.matmul(H[u][n], cp.matmul(Rxx[u][n], H[u][n].T.conj())) for u in range(U))
    I = np.eye(Ly)
    constraints.append(sum(b[u][n] for u in range(U)) <= cp.log_det(I + matrix_sum) / np.log(2))

# Diagonal elements of Rxx(u, n) are positive for all u, n
for u in range(U):
    for n in range(N):
        for i in range(Lx[u]):
            constraints.append(Rxx[u][n][i, i] >= 0)
            
# 2^U - 1 rate capacity constraints for all subsets of users
# construct the power set of U
subsets = list(powerset(range(U)))
subsets = subsets[1:]  # remove the empty set
for subset in subsets:
    # Compute the capacity bound for this subset for each FFT tone n
    for n in range(N):
        # Compute the matrix sum inside the determinant
        matrix_sum = sum(cp.matmul(H[u][n], cp.matmul(Rxx[u][n], H[u][n].T.conj())) for u in subset)
        I = np.eye(Ly)
        capacity_bound = cp.log_det(I + matrix_sum) / np.log(2)
        rate_sum = sum(b[u][n] for u in subset)
        constraints.append(rate_sum <= capacity_bound)    


In [28]:
# Objective: minimize sum of traces of Rxx(u, n)
objective = cp.Minimize(sum(sum(cp.trace(Rxx[u][n]) for n in range(N)) for u in range(U)))

In [29]:
# Problem setup
problem = cp.Problem(objective, constraints)

# Solve the problem
result = problem.solve()

# Get the dual variables
thetas = [constraint.dual_value for constraint in b_constraints]

In [30]:
print("The minimum value of the objective is:", result)
for u in range(U):
    for n in range(N):
        print(f"Rxx[{u}][{n}] =", Rxx[u][n].value)
        print(f"b[{u}][{n}] =", b[u][n].value)

The minimum value of the objective is: 27.91451226515833
Rxx[0][0] = [[3.99408184e-08]]
b[0][0] = 1.6726235859846726e-06
Rxx[0][1] = [[0.1383623]]
b[0][1] = 3.486113538708121
Rxx[0][2] = [[0.22170369]]
b[0][2] = 4.615667973445999
Rxx[0][3] = [[0.22776256]]
b[0][3] = 5.020458491730375
Rxx[0][4] = [[0.22797511]]
b[0][4] = 5.197911929208977
Rxx[0][5] = [[0.22314089]]
b[0][5] = 5.138468106638598
Rxx[0][6] = [[0.21026408]]
b[0][6] = 4.81423914983958
Rxx[0][7] = [[0.17699309]]
b[0][7] = 4.124322121625444
Rxx[0][8] = [[3.2191705e-08]]
b[0][8] = 1.461523395140926e-06
Rxx[0][9] = [[9.99322694e-09]]
b[0][9] = 2.609351804449523e-07
Rxx[0][10] = [[8.73022275e-09]]
b[0][10] = 2.0931273672856743e-07
Rxx[0][11] = [[9.23252319e-09]]
b[0][11] = 3.2166787717160884e-07
Rxx[0][12] = [[9.9064866e-09]]
b[0][12] = 5.133837085410793e-07
Rxx[0][13] = [[1.06112467e-08]]
b[0][13] = 7.181727430020917e-07
Rxx[0][14] = [[1.19574419e-08]]
b[0][14] = 9.252017545423264e-07
Rxx[0][15] = [[1.56191478e-08]]
b[0][15] = 1.

In [31]:
# print total data rate for each user
for u in range(U):
    print(f"Total data rate for user {u} is", sum(b[u][n].value for n in range(N)))
# print total energy for each user
for u in range(U):
    # TODO: ask whether to take mean or sum
    print(f"Total energy for user {u} is", sum(np.trace(Rxx[u][n].value) for n in range(N))/N)

Total data rate for user 0 is 170.1192999980531
Total data rate for user 1 is 176.81499999790827
Total data rate for user 2 is 244.5389999964186
Total energy for user 0 is 0.1080322310393918
Total energy for user 1 is 0.1269179950345394
Total energy for user 2 is 0.2012140280691678


In [32]:
# print the thetas
for u in range(U):
    print(f"Theta for user {u} is", thetas[u])
# print the duality gap
print("The duality gap is:", sum(bmin) - sum(sum(b[u][n].value for n in range(N)) for u in range(U)))

Theta for user 0 is 0.17485270809512485
Theta for user 1 is 0.17472120114002915
Theta for user 2 is 0.17472119838455893
The duality gap is: 7.619973985129036e-09


In [44]:
# take the theta, and implement chap 5 equation 5.311 to get b(u,n) by successive decoding  
# for equal thetas, handle time sharing same as john
# construct pi array where pi[u] is the decoding order for user u, which is in the ascending order of thetas
pi_inv = np.argsort(thetas)
# construct pi_inv array where pi_inv[u] is the user index of the decoding order u
pi = np.argsort(pi_inv)
print(pi_inv)
# construct empty numpy array b_achieved to store the achieved data rates
b_achieved = np.zeros((U, N))
# calculate the achieved data rates b[u,n] for each user u and each FFT tone n using the decoding order pi and the formula given below
# b_achieved[u,n] = log2(det(summation over all u which appear after u in the decoding order, including u of H(u,n)Rxx(u,n)H(u,n)^H  + Identity)) - log2(det(summation over all u which appear after u in the decoding order, not including u of H(u,n)Rxx(u,n)H(u,n)^H  + Identity))
for n in range(N):
    for u in range(U):
        matrix_sum1 = sum((np.matmul(H[pi_inv[i]][n], np.matmul(Rxx[pi_inv[i]][n].value, H[pi_inv[i]][n].T.conj()))) for i in range(u, U))
        matrix_sum2 = sum((np.matmul(H[pi_inv[i]][n], np.matmul(Rxx[pi_inv[i]][n].value, H[pi_inv[i]][n].T.conj()))) for i in range(u+1, U))
        b_achieved[pi_inv[u], n] = (np.log2(np.linalg.det(np.eye(Ly) + matrix_sum1)) - np.log2(np.linalg.det(np.eye(Ly) + matrix_sum2)))
        # b_achieved[pi_inv[u], n] = (np.log2(np.linalg.det(np.eye(Ly) + matrix_sum1)) )
# print the achieved data rates
# print the total achieved data rate for each user
for u in range(U):
    print(f"Total achieved data rate for user {u} is", sum(b_achieved[u, n] for n in range(N)))

[2 1 0]
Total achieved data rate for user 0 is 170.1210051335953
Total achieved data rate for user 1 is 218.73878183973454
Total achieved data rate for user 2 is 202.613516110959


/Users/sagnik/.pyenv/versions/3.9.6/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/sagnik/.pyenv/versions/3.9.6/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/var/folders/wd/7s_rgclx5rlc79rrjnspznh00000gn/T/ipykernel_33107/913283887.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  b_achieved[pi_inv[u], n] = (np.log2(np.linalg.det(np.eye(Ly) + matrix_sum1)) - np.log2(np.linalg.det(np.eye(Ly) + matrix_sum2)))


In [45]:
pi_inv = [1, 2, 0]
# construct pi_inv array where pi_inv[u] is the user index of the decoding order u
pi = np.argsort(pi_inv)
print(pi_inv)
# construct empty numpy array b_achieved to store the achieved data rates
b_achieved = np.zeros((U, N))
# calculate the achieved data rates b[u,n] for each user u and each FFT tone n using the decoding order pi and the formula given below
# b_achieved[u,n] = log2(det(summation over all u which appear after u in the decoding order, including u of H(u,n)Rxx(u,n)H(u,n)^H  + Identity)) - log2(det(summation over all u which appear after u in the decoding order, not including u of H(u,n)Rxx(u,n)H(u,n)^H  + Identity))
for n in range(N):
    for u in range(U):
        matrix_sum1 = sum((np.matmul(H[pi_inv[i]][n], np.matmul(Rxx[pi_inv[i]][n].value, H[pi_inv[i]][n].T.conj()))) for i in range(u, U))
        matrix_sum2 = sum((np.matmul(H[pi_inv[i]][n], np.matmul(Rxx[pi_inv[i]][n].value, H[pi_inv[i]][n].T.conj()))) for i in range(u+1, U))
        b_achieved[pi_inv[u], n] = np.log2(np.linalg.det(np.eye(Ly) + matrix_sum1)) - np.log2(np.linalg.det(np.eye(Ly) + matrix_sum2))
# print the achieved data rates
# print the total achieved data rate for each user
for u in range(U):
    print(f"Total achieved data rate for user {u} is", sum(b_achieved[u, n] for n in range(N)))

[1, 2, 0]
Total achieved data rate for user 0 is 170.1210051335953
Total achieved data rate for user 1 is 95.89119375350762
Total achieved data rate for user 2 is 325.46110419718605


/Users/sagnik/.pyenv/versions/3.9.6/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/sagnik/.pyenv/versions/3.9.6/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/var/folders/wd/7s_rgclx5rlc79rrjnspznh00000gn/T/ipykernel_33107/2419738453.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  b_achieved[pi_inv[u], n] = np.log2(np.linalg.det(np.eye(Ly) + matrix_sum1)) - np.log2(np.linalg.det(np.eye(Ly) + matrix_sum2))


In [36]:
pi_inv = [0, 1, 2]
# construct pi_inv array where pi_inv[u] is the user index of the decoding order u
pi = np.argsort(pi_inv)
print(pi_inv)
# construct empty numpy array b_achieved to store the achieved data rates
b_achieved = np.zeros((U, N))
# calculate the achieved data rates b[u,n] for each user u and each FFT tone n using the decoding order pi and the formula given below
# b_achieved[u,n] = log2(det(summation over all u which appear after u in the decoding order, including u of H(u,n)Rxx(u,n)H(u,n)^H  + Identity)) - log2(det(summation over all u which appear after u in the decoding order, not including u of H(u,n)Rxx(u,n)H(u,n)^H  + Identity))
for n in range(N):
    for u in range(U):
        matrix_sum1 = sum(np.real(np.matmul(H[pi_inv[u]][n], np.matmul(Rxx[pi_inv[u]][n].value, H[pi_inv[u]][n].T.conj()))) for i in range(u, U))
        matrix_sum2 = sum(np.real(np.matmul(H[pi_inv[u]][n], np.matmul(Rxx[pi_inv[u]][n].value, H[pi_inv[u]][n].T.conj()))) for i in range(u+1, U))
        b_achieved[pi_inv[u], n] = np.log2(np.linalg.det(np.eye(Ly) + matrix_sum1)) - np.log2(np.linalg.det(np.eye(Ly) + matrix_sum2))
# print the achieved data rates
# print the total achieved data rate for each user
for u in range(U):
    print(f"Total achieved data rate for user {u} is", sum(b_achieved[u, n] for n in range(N)))

[0, 1, 2]
Total achieved data rate for user 0 is 35.5262496451889
Total achieved data rate for user 1 is 71.24791436816172
Total achieved data rate for user 2 is 543.064821041217
